## Another Classification exercise

We will use the "Social_Network_Ads" dataset for this exercise. 
The dataset is on social network ads and users' click-through. 

We want to build a model to predict if a new user will click on an ad.

##Data preparation


In [ ]:
# make sure to upload the dataset to your colab folder before proceeding (see instructions in "Getting Started with Colab")
#loading the 50_startups datasets as a panda dataframe
import pandas as pd
df = pd.read_csv('Social_Network_Ads.csv')
df.describe()

In [ ]:
# number of users who Clicked on the ad
df['Clicked'].sum()

In [ ]:
#visually explore the data: correlation matrix of the dataset
import seaborn as sb
cormatrix=df.corr()
sb.heatmap(cormatrix,cmap='YlGn') 
#cmap(colormap) options: 'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds','YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu','GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'

In [ ]:
##preparing the data for model training
y = df[['Clicked']]  
X = df[['Age','EstimatedSalary']] 

None of the potential predictor/input/X variables are categorical, so we won't need to encode them (i.e., dummy or OneHot encoding). 

In [ ]:
X.head(2)

In [ ]:
y.head(2)

## Splitting the data into Train/Test set
We will split the data intro train and test set, try several classifications models, and evaluate their predicitve performance:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#splitting the dataset into training (70%) and testing (30%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#uncomment ONLY one the following two, depending on how you want to scale the data
#sc = StandardScaler() #standardization
sc = MinMaxScaler() #minmax scaling
X_train_sc=sc.fit_transform(X_train)

In [ ]:
# Let's compare the histograms of pre/post scaling for one variable ("mean radius") in our training data (X_train)
import matplotlib.pyplot as plt
fig, axs = plt.subplots(ncols=2)
sb.distplot(X_train[X_train.columns[1]], ax=axs[0])
sb.distplot(X_train_sc[[1]], ax=axs[1])


##Logistic Regression

In [ ]:
#using logistic regression 
from sklearn.linear_model import LogisticRegression
logistic1 = LogisticRegression(random_state = 0, solver='liblinear')
logistic1.fit(X_train_sc, y_train) #fitting the model

In [ ]:
#using the trained model to predict outcome values for the test data (X_test)
y_pred = logistic1.predict(sc.transform(X_test))


##Model Evaluation
we can use sklearn.metrics to calculate performance metrics (the code is provided by commented out).

Another option is scikitplot (which runs on top of sklearn) to create more appealing display for evaluation metrics such as the confusion matrix and various model evaluation plots.

In [ ]:
!pip install scikit-plot #run once to install library

In [ ]:
#!pip install scikit-plot #run once to install library
#using scikit plot for model evaluation
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_test, y_pred) #confusion matrix

In [ ]:
#Let's take a look at model's Precision, Recall, and Accuracy
(metrics.precision_score(y_test, y_pred), #Precision
metrics.recall_score(y_test, y_pred), #Recall
metrics.accuracy_score(y_test, y_pred), #Accuracy
metrics.balanced_accuracy_score(y_test, y_pred)) #Balanced Accuracy

In [ ]:
#calculate probabilty of each observation in X-test to belong to either outcome class; Benign=0 , Malignant=1
y_pred_prob=logistic1.predict_proba(sc.fit_transform(X_test))
#y_pred_prob

In [ ]:
#plotting the ROC curve to evalute the model's aggregate performance
skplt.metrics.plot_roc_curve(y_test,y_pred_prob)


In [ ]:
# plotting the precision-recall curve
skplt.metrics.plot_precision_recall_curve(y_test,y_pred_prob)

##Decision Tree Classification 

Learn more at https://scikit-learn.org/stable/modules/tree.html#tree 

In [ ]:
#using the Decision Tree Classifier from sklearn
from sklearn.tree import DecisionTreeClassifier
DecTree1 = DecisionTreeClassifier(criterion = 'entropy', random_state = 123) #change metric to 'gini' and rerun
DecTree1.fit(X_train, y_train)

In [ ]:
#using the model to predict outcomes for X_test
y_pred = DecTree1.predict(X_test) 
#confusion matrix for the DecTree Classifier
skplt.metrics.plot_confusion_matrix(y_test, y_pred)
#calculating other model evalution metrics
(metrics.precision_score(y_test, y_pred), #Precision
metrics.recall_score(y_test, y_pred), #Recall
metrics.accuracy_score(y_test, y_pred)) #Accuracy

Change the split criterion for the Decision Tree to 'gini' and rerun the model and evalution. 
###Question
Does it improve the model? which metric did you consider for your answer?

## Cross validation 
Is an approach to evalute machine learning models. CV helps with **overfitting** in supervised learning. It is also use to for model selection (selecting the best performing model between several candidates).

https://scikit-learn.org/stable/modules/cross_validation.html 


##Comparing several models (using CV for model validation)
Now we will put everything together and compare several models based on several performance metrics. We use k-fold cross-validation to evaluate each model. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# prepare models
models = []
models.append(('LogisticReg', LogisticRegression(max_iter=500)))
models.append(('K-NN      ', KNeighborsClassifier()))
models.append(('DecisTree', DecisionTreeClassifier(criterion = 'entropy')))
models.append(('NaiveBayes', GaussianNB()))
models.append(('SVM       ', SVC())) # add a SVM classifieer
# the next line adds a RandomForest classifier
models.append(('RandForest', RandomForestClassifier(n_estimators = 10,criterion = 'entropy')))
# evaluate each model in turn
results = []
names = []
scoring = 'precision' #metric we want to compare
#see https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter for complete list of options
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=123)
	cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison: ' +scoring)
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
df.columns

### Question
Which model has the best Accuracy? 

change the scoring metric in the previous code cell as needed to answer the following questions. For example: 
* scoring='accuracy'
* scoring='precision'
* scoring='recall'

see https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter for a comprehensive list


### Question 
Which model has the best Precision? best Recall?

## Interactive Visualization of Decision Tree Classifers

In [ ]:
!pip install ipywidgets

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display                               
from ipywidgets import interactive
# class labels
labels = X.columns #
def plot_tree(crit, split, depth, min_split, min_leaf=0.2):
  estimator = DecisionTreeClassifier(random_state = 0, criterion = crit, 
                                     splitter = split, max_depth = depth,
                                     min_samples_split=min_split, 
                                     min_samples_leaf=min_leaf)
  estimator.fit(X, y)
  graph = Source(tree.export_graphviz(estimator, out_file=None, 
                                      feature_names=labels, 
                                      class_names=['0', '1', '2'], filled = True))
  display(SVG(graph.pipe(format='svg')))
  return estimator

inter=interactive(plot_tree, 
                  crit = ["gini", "entropy"] , 
                  split = ["best", "random"] , 
                  depth=[1,2,3,4], 
                  min_split=(0.1,1), #min number of samples to further split a node
                  min_leaf=(0.1,0.5)) #min number of samples required to be a leaf node
display(inter)

###Question
What is the most important variable to consider to predict whether a used will click on the ad?